In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from functools import partial

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizer, AutoTokenizer
from transformer_lens import HookedTransformer
import torch.nn.functional as F

In [3]:
from eap.graph import Graph
from eap.evaluate import evaluate_graph, evaluate_baseline
from eap.attribute import attribute 

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

def load_model(adapter_path, hf_model_name, translens_model_name, scratch_cache_dir = None):
    base_model = AutoModelForCausalLM.from_pretrained(hf_model_name, cache_dir=scratch_cache_dir)
    model_with_lora = PeftModel.from_pretrained(base_model, adapter_path)
    model_with_lora = model_with_lora.merge_and_unload()
    model = HookedTransformer.from_pretrained(model_name=translens_model_name, hf_model=model_with_lora, cache_dir=scratch_cache_dir)  

    model.cfg.use_split_qkv_input = True
    model.cfg.use_attn_result = True
    model.cfg.use_hook_mlp_in = True
    model.cfg.ungroup_grouped_query_attention = True
    return model

In [5]:
def collate_EAP(xs):
    clean, corrupted, labels = zip(*xs)
    clean = list(clean)
    corrupted = list(corrupted)
    return clean, corrupted, labels

class EAPDataset(Dataset):
    def __init__(self, filepath):
        self.df = pd.read_csv(filepath)

    def __len__(self):
        return len(self.df)
    
    def shuffle(self):
        self.df = self.df.sample(frac=1)

    def head(self, n: int):
        self.df = self.df.head(n)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        return row['clean'], row['corrupted'], row['answer']
    
    def to_dataloader(self, batch_size: int):
        return DataLoader(self, batch_size=batch_size, collate_fn=collate_EAP)
    
def get_logit_positions(logits: torch.Tensor, input_length: torch.Tensor):
    batch_size = logits.size(0)
    idx = torch.arange(batch_size, device=logits.device)

    logits = logits[idx, input_length - 1]
    return logits

def kl_divergence(logits: torch.Tensor, clean_logits: torch.Tensor, input_length: torch.Tensor, labels: torch.Tensor, mean=True, loss=True):
    logits = get_logit_positions(logits, input_length)
    clean_logits = get_logit_positions(clean_logits, input_length)

    probs = torch.softmax(logits, dim=-1)
    clean_probs = torch.softmax(clean_logits, dim=-1)
    results = F.kl_div(probs.log(), clean_probs.log(), log_target=True, reduction='none').mean(-1)
    return results.mean() if mean else results

In [6]:
scratch_cache_dir = "/mnt/faster0/rje41/.cache/huggingface"    
hf_model_name = "EleutherAI/pythia-1.4B-deduped"
translens_model_name="pythia-1.4B-deduped"
adapter_path = "/homes/rje41/mech-interp-ft/circuit-analysis/prompt_variation/results/checkpoint-300/prompt_template_0/graph.json"
model = load_model(
        adapter_path=adapter_path,
        hf_model_name=hf_model_name,
        translens_model_name=translens_model_name,
        scratch_cache_dir=scratch_cache_dir,
    )

ValueError: Can't find 'adapter_config.json' at '/homes/rje41/mech-interp-ft/circuit-analysis/prompt_variation/results/checkpoint-300/prompt_template_0/graph.json'

In [ ]:
graph = 

In [ ]:
ds = EAPDataset('arithmetic-bases/Base10/datasets_csv/validation.csv')
dataloader = ds.to_dataloader(6)

In [27]:
def exact_match_accuracy(model, logits, corrupted_logits, input_lengths, labels):
    batch_size = logits.size(0)
    device = logits.device
    positions = input_lengths - 1

    last_logits = logits[torch.arange(batch_size), positions, :]
    predicted_tokens = last_logits.argmax(dim=-1)
    predicted_strings = [model.to_string(token.item()).strip() for token in predicted_tokens]

    labels_strings = []
    for i in range(batch_size):
        lab = labels[i]
        if isinstance(lab, torch.Tensor):
            lab = lab.item()
        labels_strings.append(str(lab).strip())

    correct = []
    for pred_str, label_str in zip(predicted_strings, labels_strings):
        print(f'Pred_str: {pred_str}, label_str: {label_str}')
        if pred_str == label_str:
            correct.append(1.0)
        else:
            correct.append(0.0)

    return torch.tensor(correct, device=device)

def calculate_accuracy(model, g, dataloader):
    baseline_accuracy = evaluate_baseline(model, dataloader, partial(exact_match_accuracy, model)).mean().item()
    graph_accuracy = evaluate_graph(model, g, dataloader, partial(exact_match_accuracy, model)).mean().item()   
    return baseline_accuracy, graph_accuracy

baseline_accuracy = evaluate_baseline(model, dataloader, partial(exact_match_accuracy, model)).mean().item()
print(baseline_accuracy)
# graph_accuracy = evaluate_graph(model, g, dataloader, partial(exact_match_accuracy, model)).mean().item() 

  1%|          | 1/100 [00:00<00:56,  1.74it/s]

Pred_str: 369, label_str: 369
Pred_str: 410, label_str: 410
Pred_str: 776, label_str: 770
Pred_str: 996, label_str: 996
Pred_str: 351, label_str: 351
Pred_str: 802, label_str: 802


  2%|▏         | 2/100 [00:01<00:56,  1.73it/s]

Pred_str: 532, label_str: 532
Pred_str: 763, label_str: 763
Pred_str: 1934, label_str: 1914
Pred_str: 880, label_str: 880
Pred_str: 609, label_str: 609
Pred_str: 847, label_str: 847


  3%|▎         | 3/100 [00:01<00:58,  1.67it/s]

...

Pred_str: 924, label_str: 924
Pred_str: 411, label_str: 421
Pred_str: 838, label_str: 838
Pred_str: 862, label_str: 862
Pred_str: 528, label_str: 528
Pred_str: 279, label_str: 279


 99%|█████████▉| 99/100 [00:52<00:00,  1.91it/s]

Pred_str: 790, label_str: 790
Pred_str: 431, label_str: 431
Pred_str: 482, label_str: 482
Pred_str: 543, label_str: 543
Pred_str: 907, label_str: 907
Pred_str: 808, label_str: 808


100%|██████████| 100/100 [00:53<00:00,  1.88it/s]

Pred_str: 987, label_str: 987
Pred_str: 549, label_str: 549
Pred_str: 718, label_str: 718
Pred_str: 860, label_str: 860
Pred_str: 679, label_str: 679
Pred_str: 633, label_str: 643
0.8050000071525574
